A amostra de dados em anexo (info_transportes.csv), possui dados de um
aplicativo de transporte privado, cujas colunas são:
* DATA_INICIO (formato: "mm-dd-yyyy HH")
* DATA_FIM (formato: "mm-dd-yyyy HH")
* CATEGORIA
* LOCAL_INICIO
* LOCAL_FIM
* PROPOSITO
* DISTANCIA

<br>
Uma equipe está elaborando um modelo para compreender como os clientes
estão utilizando o aplicativo. Para isso, você precisa fornecer uma nova tabela
“info_corridas_do_dia", com dados agrupados pela data de início do transporte
utilizando a formatação “yyyy-MM-dd”, contendo as seguintes colunas:

| Nome da Coluna | Descrição |
   |---|---|
   | DT_REFE | Data de referência. |
   | QT_CORR | Quantidade de corridas. |
   | QT_CORR_NEG | Quantidade de corridas com a categoria “Negócio”. |
   | QT_CORR_PESS | Quantidade de corridas com a categoria “Pessoal”. |
   | VL_MAX_DIST | Maior distância percorrida por uma corrida. |
   | VL_MIN_DIST | Menor distância percorrida por uma corrida. |
   | VL_AVG_DIST | Média das distâncias percorridas. |
   | QT_CORR_REUNI | Quantidade de corridas com o propósito de "Reunião". |
   | QT_CORR_NAO_REUNI | Quantidade de corridas com o propósito declarado e diferente de "Reunião". |
   
<br>
Abaixo temos um exemplo de uma possível linha da tabela:

| DT_REFE | QT_CORR | QT_CORR_NEG | QT_CORR_PESS | VL_MAX_DIST | VL_MIN_DIST | VL_AVG_DIST | QT_CORR_REUNI | QT_CORR_NAO_REUNI |
   |---|---|---|---|---|---|---|---|---|
   | 2022-01-01 | 20 | 12 | 8 | 2.2 | 0.7 | 1.1 | 6 | 10 |

O código que você irá elaborar para criar a tabela (ou dataframe spark/pandas)
deverá ser escrito em uma das seguintes linguagens: SQL ou Python (pyspark
e/ou pandas). Caso decida por SQL, considere o nome da tabela como "
info_transportes".

<h3>Exploração de Dados com Pandas</h3>

In [1]:
import pandas as pd
import pyodbc

In [113]:
df = pd.read_csv('\\projetos\info_transportes.csv', sep=';')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1153 entries, 0 to 1152
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DATA_INICIO   1153 non-null   object
 1   DATA_FIM      1153 non-null   object
 2   CATEGORIA     1153 non-null   object
 3   LOCAL_INICIO  1153 non-null   object
 4   LOCAL_FIM     1153 non-null   object
 5   DISTANCIA     1153 non-null   int64 
 6   PROPOSITO     651 non-null    object
dtypes: int64(1), object(6)
memory usage: 63.2+ KB


In [4]:
df.head()

,DATA_INICIO,DATA_FIM,CATEGORIA,LOCAL_INICIO,LOCAL_FIM,DISTANCIA,PROPOSITO
0,01-01-2016 21:11,01-01-2016 21:17,Negocio,Fort Pierce,Fort Pierce,51,Alimentação
1,01-02-2016 01:25,01-02-2016 01:37,Negocio,Fort Pierce,Fort Pierce,5,NaN
2,01-02-2016 20:25,01-02-2016 20:38,Negocio,Fort Pierce,Fort Pierce,48,Entregas
3,01-05-2016 17:31,01-05-2016 17:45,Negocio,Fort Pierce,Fort Pierce,47,Reunião
4,01-06-2016 14:42,01-06-2016 15:49,Negocio,Fort Pierce,West Palm Beach,637,Visita ao cliente


<h3>Limpeza da Base</h3>

In [49]:
df.DATA_INICIO.str[:10]

0       01-01-2016
1       01-02-2016
2       01-02-2016
3       01-05-2016
4       01-06-2016
           ...    
1148    12-31-2016
1149    12-31-2016
1150    12-31-2016
1151    12-31-2016
1152    12-31-2016
Name: DATA_INICIO, Length: 1153, dtype: object

In [114]:
df['DATA_INICIO'] = pd.to_datetime(df.DATA_INICIO.str[6:10] + '-' + df.DATA_INICIO.str[:2] + '-' + df.DATA_INICIO.str[3:5], format='%Y-%m-%d')
df['DATA_FIM'] = pd.to_datetime(df.DATA_FIM.str[6:10] + '-' + df.DATA_FIM.str[:2] + '-' + df.DATA_FIM.str[3:5], format='%Y-%m-%d')

In [115]:
df['PROPOSITO'] = df['PROPOSITO'].fillna('não definido')

In [116]:
df['DT_REFE'] = df['DATA_FIM']

<h3>Transferência para o SQL Server</h3>

In [117]:
df

,DATA_INICIO,DATA_FIM,CATEGORIA,LOCAL_INICIO,LOCAL_FIM,DISTANCIA,PROPOSITO,DT_REFE
0,2016-01-01,2016-01-01,Negocio,Fort Pierce,Fort Pierce,51,Alimentação,2016-01-01
1,2016-01-02,2016-01-02,Negocio,Fort Pierce,Fort Pierce,5,não definido,2016-01-02
2,2016-01-02,2016-01-02,Negocio,Fort Pierce,Fort Pierce,48,Entregas,2016-01-02
3,2016-01-05,2016-01-05,Negocio,Fort Pierce,Fort Pierce,47,Reunião,2016-01-05
4,2016-01-06,2016-01-06,Negocio,Fort Pierce,West Palm Beach,637,Visita ao cliente,2016-01-06
...,...,...,...,...,...,...,...,...
1148,2016-12-31,2016-12-31,Negocio,Karchi,Karchi,7,Reunião,2016-12-31
1149,2016-12-31,2016-12-31,Negocio,Karchi,Unknown Location,39,Parada temporária,2016-12-31
1150,2016-12-31,2016-12-31,Negocio,Unknown Location,Unknown Location,162,Reunião,2016-12-31
1151,2016-12-31,2016-12-31,Negocio,Katunayake,Gampaha,64,Parada temporária,2016-12-31


In [ ]:
conexao_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=(localdb)\\LocalDBDemo;DATABASE=BD_FIN_PES;Trusted_Connection=yes'
conexao = pyodbc.connect(conexao_string)

nome_da_tabela = 'info_transportes'
df.to_sql(nome_da_tabela, con=conexao, if_exists='replace', index=False)